In [1]:
from pydrake.all import (
    LeafSystem,
    MultibodyPlantConfig,
    Meshcat,
    StartMeshcat,
    DiagramBuilder,
    AddMultibodyPlant,
    Parser,
    LoadModelDirectives,
    ModelDirectives,
    ProcessModelDirectives,
    Diagram,
    Context,
    AddDefaultVisualization,
    RigidTransform,
    Quaternion
)
import numpy as np
import matplotlib.pyplot as plt

In [2]:
meshcat: Meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


## Example Setup for Spinner

In [3]:
# Setup 
config = MultibodyPlantConfig()
config.time_step = 0.001
config.penetration_allowance = 0.001
config.contact_model = "hydroelastic_with_fallback"
config.contact_surface_representation = "polygon"
config.discrete_contact_approximation = "tamsi"

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlant(config, builder)
parser = Parser(plant, scene_graph)
parser.package_map().AddPackageXml('../package.xml')

directive_file = "../config/spinner.yaml"
directives: ModelDirectives = LoadModelDirectives(directive_file)
models = ProcessModelDirectives(directives, plant, parser)
plant.Finalize()

AddDefaultVisualization(builder, meshcat)

diagram: Diagram = builder.Build()
context: Context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
diagram.ForcedPublish(context)

In [5]:
context.SetTime(0.0)
plant.SetPositions(plant_context, [0.2, 0.2, 1.0])
body_rt = RigidTransform(Quaternion(1.0,0.0,0.0,0.0), [0.0, 0.0, 0.25])
diagram.ForcedPublish(context)

Playing around with this setup, you'll notice that the first two positions in the plant are related to the two revolute jointed arm. The arm is a fixed base (meaning it's welded to the floor at the origin).

The ball is also a fixed base with only rotated being possible. This is planned in SE(2) as a small test for contact implicit MPC.